# Meta data overview

In [1]:
from fdsobj import FdsObj
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os

datadir = "raw/"

In [2]:
fname = '058_Weight_Car2_5_passenger_60kmh_West_And_East_2020.02.26.03.23.01.498982_20210201T185536_OPD.fds'
fname = os.path.join(datadir, fname)

fds = FdsObj(fname)
fds

FDS File: 058_Weight_Car2_5_passenger_60kmh_West_And_East_2020.02.26.03.23.01.498982_20210201T185536_OPD.fds
data_type: real32
start_bin: 0
end_bin: 1027
start_shot: 0 
sample_rate_Hz: 1000.04
bin_spacing_m: 0.68063
block_size: 1

In [3]:
def reload_data(fname, numShots):
    fds = FdsObj(fname)
    data = np.asarray([shot for _,shot in zip(range(numShots), fds)])
    data = np.transpose(data)
    return fds, data #datashape: (bin, shot)

In [4]:
numShots = 10000000
fds, data = reload_data(fname, numShots)

In [5]:
print(data.shape)

(1028, 56096)


# generate data

In [17]:
speed = "058"
numShots = data.shape[1]
bins = np.array([350, 400,450, 500,550, 600,650, 690])
start_bin, end_bin = bins[0], bins[-1]
directions  = ["west"]


In [18]:
from utils.data_preprocessing import *

In [19]:
fds, data = reload_data(fname, numShots)
output_labels = np.zeros((data.shape[0], data.shape[1])) #create placeholder output_labels
get_all_cars_window_width = []

In [20]:
car_label = 1 # with 5p in car 2 => label = 1
print("="*40)
print("Process {0} car".format(car_label))
d = "west"
print("Direction {0}".format(d))

shot_start, shot_end = get_start_end_shot(str(car_label), speed, d)
a1, b1, a2, b2 = get_a1b1_a2b2(bins, shot_start, shot_end) # get slopes
check_slope(a1, a2, car_label, d)
output_labels, window_width_list = get_labels(output_labels, start_bin, end_bin, a1, b1, a2, b2, car_label, 520, 570)
get_all_cars_window_width.extend(window_width_list)

Process 1 car
Direction west


In [21]:
import os 
save_root_folder = 'preprocessed_data/'
if not os.path.exists(save_root_folder):
    os.mkdir(save_root_folder)


output_fname_x, output_fname_y = save_root_folder+'058_5p_X.txt', save_root_folder+'058_5p_y.txt'
car_unique_labels = [1] #car2 + 5p => HOV occupancy: label=1 
biggest_window_len = 5976 # we will pad all signals to this length
export_data_v1(data, output_labels, start_bin, end_bin, biggest_window_len, output_fname_x, output_fname_y)